In [ ]:
SELECT *
FROM SNOWFLAKE_LEARNING_DB.SAMMATHEWSON98_LOAD_SAMPLE_DATA_FROM_S3.HEALTH_CLAIMS_PARQUET
LIMIT 10;


In [ ]:
SELECT CURRENT_ROLE(), CURRENT_DATABASE(), CURRENT_SCHEMA(), CURRENT_WAREHOUSE();

In [ ]:
SHOW TABLES;

In [ ]:
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import functions as F
import re

session = get_active_session()

DB = "SNOWFLAKE_LEARNING_DB"
SCHEMA = "SAMMATHEWSON98_LOAD_SAMPLE_DATA_FROM_S3"

RAW_TABLE = f"{DB}.{SCHEMA}.HEALTH_CLAIMS_PARQUET"         # raw table with 1 column of JSON-ish text
OUT_TABLE = f"{DB}.{SCHEMA}.HEALTH_CLAIMS_TEMP"     # output: one column per JSON key

def normalize_col(name: str) -> str:
    s = (name or "").strip().lower()
    s = re.sub(r"[^a-z0-9_]", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    if not s:
        s = "col"
    if s[0].isdigit():
        s = f"c_{s}"
    return s

df_raw = session.table(RAW_TABLE)

# Assume the raw payload is in the first column (common for these loads)
raw_col = df_raw.columns[0]
raw = F.col(raw_col)

# Your sample looked like: "{   ""claim_id"": ""..."" }"
# So: trim outer quotes, replace doubled quotes "" -> ", then parse
cleaned = F.replace(F.trim(raw, F.lit('"')), F.lit('""'), F.lit('"'))
j = F.try_parse_json(cleaned)

df_parsed = df_raw.select(j.alias("J")).filter(F.col("J").is_not_null()).limit(20000)

# ---- Discover keys dynamically via FLATTEN (alias ALL 6 output columns) ----
flat = F.flatten(F.col("J")).as_("SEQ", "KEY", "PATH", "INDEX", "VALUE", "THIS")

keys_rows = (
    df_parsed
    .select(flat)
    .select(F.col("KEY").as_("KEY"))
    .distinct()
    .collect()
)

keys = [r["KEY"] for r in keys_rows if r["KEY"] is not None]
if not keys:
    raise ValueError("No JSON keys discovered. Parsing may be failing; inspect the raw values.")

# ---- Build select list (one output column per key) ----
used = {}
select_exprs = []

for k in keys:
    base = normalize_col(str(k))
    colname = base

    # Ensure unique names after normalization
    if colname in used:
        used[colname] += 1
        colname = f"{base}_{used[base]}"
    else:
        used[colname] = 1

    # STRING-first is safest for auto-schema.
    # If you prefer VARIANT columns instead, replace to_varchar(...) with F.col("J")[k]
    select_exprs.append(F.regexp_replace(
    F.to_varchar(F.col("J")[k]),
    r'^"|"$',
    ''
).as_(colname)
)

df_out = df_parsed.select(*select_exprs)

# Write table
df_out.write.mode("overwrite").save_as_table(OUT_TABLE)

session.table(OUT_TABLE).limit(10).show()
print(f"Created {OUT_TABLE} with {len(select_exprs)} columns.")

In [ ]:
CREATE OR REPLACE TABLE HEALTH_CLAIMS AS
SELECT
  /* identifiers + categories (keep as strings) */
  NULLIF(CLAIM_ID, '')                    AS CLAIM_ID,
  NULLIF(PATIENT_ID, '')                  AS PATIENT_ID,
  NULLIF(CPT, '')                         AS CPT,
  NULLIF(DIAGNOSIS_CODE, '')              AS DIAGNOSIS_CODE,
  NULLIF(ICD10CM_CODE_DESCRIPTION, '')    AS ICD10CM_CODE_DESCRIPTION,
  NULLIF(DIAGNOSIS_QUAL, '')              AS DIAGNOSIS_QUAL,
  NULLIF(BENEFIT_TYPE, '')                AS BENEFIT_TYPE,
  NULLIF(PAY_TYPE, '')                    AS PAY_TYPE,
  NULLIF(LOCATION_OF_CARE, '')            AS LOCATION_OF_CARE,
  NULLIF(NPI_ROLE, '')                    AS NPI_ROLE,
  NULLIF(MEMBER_MATCH, '')                AS MEMBER_MATCH,
  NULLIF(ADMIT_DIAGNOSIS_IND, '')         AS ADMIT_DIAGNOSIS_IND,
  NULLIF(PATIENT_STATE, '')               AS PATIENT_STATE,
  NULLIF(PATIENT_GENDER, '')              AS PATIENT_GENDER,
  NULLIF(PROCEDURE_MODIFIER1, '')         AS PROCEDURE_MODIFIER1,
  NULLIF(RENDERING_TAXONOMY_CODE, '')     AS RENDERING_TAXONOMY_CODE,
  NULLIF(BILLING_TAXONOMY_CODE, '')       AS BILLING_TAXONOMY_CODE,
  NULLIF(REFERRING_TAXONOMY_CODE, '')     AS REFERRING_TAXONOMY_CODE,

  /* NPIs + ZIP: keep as strings (and clean a trailing .0 if it exists) */
  REGEXP_REPLACE(NULLIF(RENDERING_NPI, ''), '\\.0$', '')  AS RENDERING_NPI,
  REGEXP_REPLACE(NULLIF(BILLING_NPI, ''),   '\\.0$', '')  AS BILLING_NPI,
  REGEXP_REPLACE(NULLIF(REFERRING_NPI, ''), '\\.0$', '')  AS REFERRING_NPI,

  /* dates */
  TRY_TO_DATE(NULLIF(DATE_SERVICE, ''), 'YYYY-MM-DD')      AS DATE_SERVICE,
  TRY_TO_DATE(NULLIF(DATE_SERVICE_END, ''), 'YYYY-MM-DD')  AS DATE_SERVICE_END,
  TRY_TO_DATE(NULLIF(DATE_START, ''), 'YYYY-MM-DD')        AS DATE_START,
  TRY_TO_DATE(NULLIF(DATE_END, ''), 'YYYY-MM-DD')          AS DATE_END,

  /* numerics */
  TRY_TO_DECIMAL(NULLIF(LINE_ALLOWED, ''), 18, 6)          AS LINE_ALLOWED,
  TRY_TO_DECIMAL(NULLIF(LINE_CHARGE, ''),  18, 6)          AS LINE_CHARGE,
  TRY_TO_DECIMAL(NULLIF(PROCEDURE_UNITS, ''), 18, 6)       AS PROCEDURE_UNITS,
  TRY_TO_NUMBER(NULLIF(PATIENT_YEAR_OF_BIRTH, ''))::INT    AS PATIENT_YEAR_OF_BIRTH,
  TRY_TO_NUMBER(NULLIF(SERVICE_LINE_NUMBER, ''))::INT      AS SERVICE_LINE_NUMBER,
  TRY_TO_NUMBER(PATIENT_ZIP3, '')                                AS PATIENT_ZIP3,

FROM HEALTH_CLAIMS_TEMP;